** RNN to generate tweets, using character level generation. **

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, RNN, Softmax, Flatten, Dropout, Input

In [9]:
data = pd.read_csv("../Load_Tweets/data/tweet_data.csv") # this will break if this file is moved!

In [10]:
data.head()

,ID,RETWEET,TEXT
0,786204978629185536,False,PAY TO PLAY POLITICS. #CrookedHillary [URL]
1,786201435486781440,False,Very little pick-up by the dishonest media of ...
2,786189446274248704,False,Crooked Hillary Clinton likes to talk about th...
3,786054986534969344,False,Thank you Florida- a MOVEMENT that has never b...
4,786007502639038464,False,Join me Thursday in Florida &amp; Ohio!West Pa...


In [25]:
# Put all the tweets into one string.

tweet_txt = data['TEXT'].str.cat(sep=' ')


In [26]:
# Ok, let's check out one of these tweets.

print(tweet_txt[:150])

PAY TO PLAY POLITICS. #CrookedHillary [URL] Very little pick-up by the dishonest media of incredible information provided by WikiLeaks. So dishonest! 


In [48]:
# Get all the unique characters used.

char = list(set(tweet_txt))
char.sort()
print("Number of unique characters: ", len(char))

Number of unique characters:  369


In [50]:
# Make a character mapping.

char_to_index = dict((c, i) for i, c in enumerate(char))
index_to_char = dict((i, c) for i, c in enumerate(char))

{' ': 0,
 '!': 1,
 '"': 2,
 '#': 3,
 '$': 4,
 '%': 5,
 '&': 6,
 "'": 7,
 '(': 8,
 ')': 9,
 '*': 10,
 '+': 11,
 ',': 12,
 '-': 13,
 '.': 14,
 '/': 15,
 '0': 16,
 '1': 17,
 '2': 18,
 '3': 19,
 '4': 20,
 '5': 21,
 '6': 22,
 '7': 23,
 '8': 24,
 '9': 25,
 ':': 26,
 ';': 27,
 '=': 28,
 '?': 29,
 '@': 30,
 'A': 31,
 'B': 32,
 'C': 33,
 'D': 34,
 'E': 35,
 'F': 36,
 'G': 37,
 'H': 38,
 'I': 39,
 'J': 40,
 'K': 41,
 'L': 42,
 'M': 43,
 'N': 44,
 'O': 45,
 'P': 46,
 'Q': 47,
 'R': 48,
 'S': 49,
 'T': 50,
 'U': 51,
 'V': 52,
 'W': 53,
 'X': 54,
 'Y': 55,
 'Z': 56,
 '[': 57,
 '\\': 58,
 ']': 59,
 '_': 60,
 'a': 61,
 'b': 62,
 'c': 63,
 'd': 64,
 'e': 65,
 'f': 66,
 'g': 67,
 'h': 68,
 'i': 69,
 'j': 70,
 'k': 71,
 'l': 72,
 'm': 73,
 'n': 74,
 'o': 75,
 'p': 76,
 'q': 77,
 'r': 78,
 's': 79,
 't': 80,
 'u': 81,
 'v': 82,
 'w': 83,
 'x': 84,
 'y': 85,
 'z': 86,
 '{': 87,
 '|': 88,
 '}': 89,
 '~': 90,
 '\xa0': 91,
 '¯': 92,
 'É': 93,
 'á': 94,
 'â': 95,
 'é': 96,
 'í': 97,
 'ñ': 98,
 'ó': 99,
 'ø': 

In [7]:

# embedding_layer= Embedding(total_words, EMBEDDING_DIM, weights=[embedding_matrix],input_length=max_seq,trainable=False)
# sequence_input = Input(shape=(max_seq,), dtype='int32')
# embedded_sequences= embedding_layer(sequence_input)
model=Sequential()
# e=Embedding(total_words, EMBEDDING_DIM, weights=[embedding_matrix],input_length=maxlen,trainable=False)
# model.add(e)
model.add(LSTM(100, input_shape=(maxlen, EMBEDDING_DIM)))
# model.add(Dropout(0.1))
# model.add(Flatten())
model.add(Dense(total_words, activation="softmax"))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

NameError: name 'maxlen' is not defined